## Pair Programming ETL Carga I

In [1]:
import pandas as pd
import mysql.connector

Cread la BBDD con el nombre de energía. La BBDD tendrá la siguiente forma:
Cread las tablas de la BBDD:
Tabla fechas
Tabla nacional_renovable_no_renovable
Tabla comunidades_renovable_no_renovable
Tabla comunidades
Al final nuestra BBDD tendrá el siguiente aspecto:

BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación